# Create Panel and Combinations Databases

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import plotly.graph_objects as go
from tqdm import tqdm
import pandas as pd
from copy import copy, deepcopy

In [2]:
import gir
import gir.config as gc
from gir.dataset.panel_data import PanelData, REFERENCE
from gir.dataset.grid import Grid
from gir.ml.preprocess import get_panel_names, plane_id_list
from gir.config import DATA_PATH as GIR_DATA_PATH

fs = 96e3

In [3]:
panel_names = []

with open('panel_names.txt', 'r') as file:
    for line in file:
        panel_names.append(line.strip())

print(panel_names)

['mean_flat_cg', 'mean_flat_rfl_1', 'mean_flat_rfl_2', 'mean_flat_vj_fu', 'mean_flat_vj_ph', 'panel_0001_0', 'panel_0001_1', 'panel_0002_1', 'panel_0003_0', 'panel_0003_1', 'panel_0004_0', 'panel_0004_1', 'panel_0005_0', 'panel_0005_1', 'panel_0006_0', 'panel_0006_1', 'panel_0007_0', 'panel_0007_1', 'panel_0008_0', 'panel_0008_1', 'panel_0009_0', 'panel_0009_1', 'panel_0010_0', 'panel_0010_1', 'panel_0011_0', 'panel_0011_1', 'panel_0012_0', 'panel_0012_1', 'panel_0013_0', 'panel_0013_1', 'panel_0014_1', 'panel_0015_0', 'panel_0015_1', 'panel_0016_0', 'panel_0016_1', 'panel_0016_r180_0', 'panel_0017_0', 'panel_0017_1', 'panel_0018_0', 'panel_0018_1', 'panel_0019_0', 'panel_0019_1', 'panel_0020_0', 'panel_0020_1', 'panel_0021_0', 'panel_0021_1', 'panel_0022_0', 'panel_0022_1', 'panel_0023_0', 'panel_0023_1', 'panel_0024_0', 'panel_0024_1', 'panel_0025_0', 'panel_0025_1', 'panel_0026_0', 'panel_0026_1', 'panel_0027_0', 'panel_0027_1', 'panel_0028_0', 'panel_0028_1', 'panel_0029_0', 'panel

## Create panel DF

In [10]:
dictlist = []

for pname in tqdm(panel_names[:]):
    panel = PanelData(pname)
    row = {
        'panel id': pname,
        'flat reference': panel.flat.panel_id,
        'foam reference': panel.foam.panel_id,
        'macrostructure reference': panel.macrostructure,
        'mesostructure reference': panel.typology,
        'sand type': panel.print_sand_type,
        'binder type': panel.print_binder_type,
    }
    dictlist.append(row)

panel_df = pd.DataFrame(dictlist)

100%|██████████| 280/280 [00:25<00:00, 10.86it/s]


In [5]:
panel_df = pd.read_csv('panel_info.csv', header=0)

In [6]:
panel_df.head(2)

,panel id,flat reference,foam reference,macrostructure reference,mesostructure reference,sand type,binder type
0,mean_flat_cg,mean_flat_cg,mean_foam_2,flat,Baseline_mean,"Quartz sand, avg grain size=190 micron",Furan binder
1,mean_flat_rfl_1,mean_flat_rfl_1,mean_foam_1,flat,Baseline_mean,"GS14 sand, avg grain size=140 micron",Phenolic binder


In [11]:
# panel_df.to_csv('panel_info.csv', index=False, header=True, mode='w')

## Create combinations DF

In [4]:
from compas.geometry import Vector, Point
from gir.dataset.grid import SURFACE_BOTTOMZ, SURFACE_TOPZ
from scipy.signal import find_peaks
grid = REFERENCE.GRID

def angle_from_cells(mcell, scell):
    """Returns angle of incidence (in deg) of sound when travelling from mcell to panel surface to scell

    Args:
        mcell (celltuple): mic cell, tuple of ints
        scell (celltuple): speaker cell, tuple of ints

    Returns:
        float: Angle of incidence, degrees.
    """
    mcellpos = copy(grid.position(mcell))
    scellpos = copy(grid.position(scell))
    mcellpos.z = abs(mcellpos.z)
    scellpos.z = abs(scellpos.z)
    scellposref = Point(scellpos.x, scellpos.y, -scellpos.z+2*SURFACE_TOPZ)
    angle_rad = (mcellpos - scellposref).angle(Vector(0, 0, 1))
    return np.abs(np.rad2deg(angle_rad))

def echo_delay_from_cells(mcell, scell, surface_top=SURFACE_TOPZ):
    """Returns echo delay (in samples, float) of sound when travelling from mcell to panel surface to scell"""
    mcellpos = copy(grid.position(mcell))
    scellpos = copy(grid.position(scell))
    mcellpos.z = abs(mcellpos.z) - surface_top
    scellpos.z = -(abs(scellpos.z) - surface_top)
    return (mcellpos - scellpos).length / 343e3 * fs

def direct_path_delay_from_cells(mcell, scell):
    """Returns direct path delay (in samples, float) of sound when travelling from mcell to scell directly"""
    mcellpos = copy(grid.position(mcell))
    scellpos = copy(grid.position(scell))
    mcellpos.z = abs(mcellpos.z)
    scellpos.z = abs(scellpos.z)
    return (mcellpos - scellpos).length / 343e3 * fs

def dp_angles(mcell, scell, surface_top=SURFACE_TOPZ):
    mcellpos: Point = copy(grid.position(mcell))
    scellpos: Point = copy(grid.position(scell))
    mcellpos.z = abs(mcellpos.z)
    scellpos.z = abs(scellpos.z)
    # get mcell's reflected point; same for scell
    mcellposref = Point(mcellpos.x, mcellpos.y, -mcellpos.z+2*surface_top)
    scellposref = Point(scellpos.x, scellpos.y, -scellpos.z+2*surface_top)
    spkr_emit_angle = (mcellpos - scellpos).angle(mcellposref - scellpos)
    mic_rec_angle = (scellpos - mcellpos).angle(scellposref - mcellpos)

    return np.abs(np.rad2deg((spkr_emit_angle, mic_rec_angle)))
    
def two_largest_peaks(ir):
    """Returns the two largest peaks in an impulse response"""
    peaks, _ = find_peaks(ir, distance=10)
    if len(peaks) < 2:
        return None
    peak_vals = ir[peaks]               # get heights
    sorted_idxs = np.argsort(peak_vals) # get indices of sorted heights
    peaks = peaks[sorted_idxs[-2:]]     # get indices of two largest peaks
    return np.sort(peaks)               # return them in order of direct path, echo path
                                        # (direct path is always first)

In [6]:
%matplotlib qt
panel = PanelData(panel_names[45])

mcell = (2, 0, 0)
scell = (3, 0, 0)
sea, mra = dp_angles(mcell, scell)
print(f'speaker emit angle: {sea}')
print(f'mic receive angle: {mra}')
ia = angle_from_cells(mcell, scell)
print(f'incidence angle: {ia}')

c = grid.combination_num_from_cells(mcell, scell)
# c = 2073
print(f'combination: {c}')

ir = panel.impulse_response_from_combination(c)
echo_delay_bottom = echo_delay_from_cells(mcell, scell, SURFACE_BOTTOMZ)
echo_delay_top = echo_delay_from_cells(mcell, scell, SURFACE_TOPZ)
echo_delay_gpy = grid.travel_distance_over_surface(mcell, scell) / 343e3 * fs
direct_path_delay = direct_path_delay_from_cells(mcell, scell)

direct_path_fp, echo_path_fp = two_largest_peaks(ir)

print(f'echo sample delay: {echo_delay_bottom}')
print(f'direct path sample delay: {direct_path_delay}')

plt.plot(ir)
plt.axvline(echo_delay_bottom, linestyle='--', color='red', label='echo (bottomz)')
plt.axvline(echo_delay_top, linestyle='--', color='green', label='echo (topz)')
plt.axvline(echo_delay_gpy, linestyle=':', color='purple', label='grid.travel_distance')
plt.axvline(echo_path_fp, linestyle=':', color='blue', label='echo path fp')

plt.axvline(direct_path_delay, linestyle=':', color='pink', label='direct path')
plt.axvline(direct_path_fp, linestyle=':', color='orange', label='direct path fp')
plt.legend()
plt.grid(True)
plt.show()

speaker emit angle: 37.11767114870481
mic receive angle: 102.44635983574976
incidence angle: 20.217984507772712
combination: 2939
echo sample delay: 235.8858238745783
direct path sample delay: 88.15771188951236


In [15]:
dictlist = []
for c in tqdm(panel.combination_numbers):
    mcell, scell = grid.cells_from_combination_num(c)
    sea, mra = dp_angles(mcell, scell)
    ia = angle_from_cells(mcell, scell)
    assert np.isclose(sea+mra+2*ia, 180, atol=1.), f'Sanity check failed for combnum {c}, panel {panel.panel_id}, sea {sea}, mra {mra}, ia {ia}'
    row = {
        'combination': c,
        'mic cell': np.array(mcell),
        'spkr cell': np.array(scell),
        'angle': np.round(ia, 2),
        'dp dly (geo)': direct_path_delay_from_cells(mcell, scell),
        'echo dly (bz)': echo_delay_from_cells(mcell, scell, SURFACE_BOTTOMZ),
        'echo dly (tz)': echo_delay_from_cells(mcell, scell, SURFACE_TOPZ),
        'echo dly (gpy)': grid.travel_distance_over_surface(mcell, scell) / 343e3 * fs,
        'spkr emit angle': np.round(sea),
        'mic rec angle': np.round(mra),
        # surface incidence angle?
    }
    row['dp echo diff'] =(row['echo dly (bz)'] + row['echo dly (tz)']) / 2 - row['dp dly (geo)']

    dictlist.append(row)

combs_df = pd.DataFrame(dictlist)

100%|██████████| 2951/2951 [00:00<00:00, 4370.91it/s]


In [16]:
combs_df.head()

,combination,mic cell,spkr cell,angle,dp dly (geo),echo dly (bz),echo dly (tz),echo dly (gpy),spkr emit angle,mic rec angle,dp echo diff
0,1,"[0, 0, 0]","[0, 0, 1]",21.54,20.991254,78.429796,57.170934,57.170934,68.0,68.0,46.809111
1,2,"[0, 0, 0]","[0, 0, 2]",38.29,41.982507,86.447273,67.752593,67.752593,52.0,52.0,35.117426
2,3,"[0, 0, 0]","[0, 0, 3]",49.82,62.973761,98.368159,82.423161,82.423161,40.0,40.0,27.421899
3,4,"[0, 0, 0]","[0, 0, 4]",57.65,83.965015,112.963374,99.388161,99.388161,32.0,32.0,22.210753
4,5,"[0, 0, 0]","[0, 0, 5]",63.13,104.956268,129.330655,117.659259,117.659259,27.0,27.0,18.538689


In [26]:
# combs_df.to_csv('combinations_df.csv', index=False, header=True, mode='w')
# to load csv:
# combinations_df = pd.read_csv(DF_PATH + '/combinations_df.csv')
combs_df.to_hdf('combinations_df.h5', key='df', mode='w')

c:\Users\Lenovo\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\generic.py:2431: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['mic cell', 'spkr cell'], dtype='object')]

  pytables.to_hdf(
